## Recurrent Neural Networks (RNN) with Keras

- Recurrent neural networks (RNN) are a class of neural networks that is powerful for modeling sequence data such as time series or natural language: e.g. speech recognition, speech synthesis, text generation, stock price prediction
- Schematically, a RNN layer uses a `for loop` to iterate over the timesteps of a sequence, while maintaining an internal state that encodes information about the timesteps it has seen so far

### with Keras

There are three built-in RNN layers in Keras:
1. [keras.layers.RNN](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN): `keras.layers.SimpleRNN`
2. [keras.layers.LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM): Long Short-Term Memory layer (Hochreiter, 1997)
3. [keras.layers.GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU): Gated Recurrent Unit (Cho et al, 2014)


## Setting up a simple RNN

In [ ]:
import numpy as np
import tensorflow as tf

from keras import Sequential
from keras.layers import SimpleRNN, Dense, Input
from tensorflow import keras
from tensorflow.keras import layers

We start with a **hypothetical example**:

- audio sequence data
- $1\,000$ audio tracks
- time dimension, e.g. max length = 30 seconds (30 time slices)
- at each time slice and for each audio track we have data on *frequency*, *decibel* (intensity), *zero-crossing rate* (number of times a waveform crosses the horizontal time axis: e.g. recognition of percussive vs pitched sounds, etc.)

The **input data shape** would be:

$$
(1000, 30, 3)
$$

1000 records x 30 seconds x 3 feature (**3D array**)

Let's try with a **dense NN**:

- for each audio sequence, we have $3 \cdot 30$ features (frequency, intensity and crossing-rate for each of 30 one-second time slices)

In [ ]:
n_sequences = 1000
n_features = 3
n_time_slices = 30
n_units = [16, 1] ## the last layers is the output (binary classification --> sigmoid)
output_activation = 'sigmoid'

In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(n_sequences, n_features*n_time_slices)))
model.add(tf.keras.layers.Dense(n_units[0], activation='relu'))
model.add(Dense(n_units[1], activation=output_activation))

model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

- (90 features x 16 units) + 16 bias_terms = 1456
- 16 unit_outputs + 1 bias_term = 17

And now with a **simple RNN model**:

In [ ]:
rnn_activation = 'tanh'
input_shape=(n_time_slices,n_features)

# SimpleRNN model
model = Sequential()
model.add(Input(input_shape))
model.add(SimpleRNN(units=n_units[0], activation=rnn_activation))
model.add(Dense(n_units[1], activation=output_activation))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.summary()

#### Calculating number of parameters to train

We have **16 units** (**+ 1 output unit**) and **3 features** which enter in the calculations for the number of trainable parameters

$$
\mathbf{W_u} \left[ \mathbf{WA^{<t-1>}_{(u,u)}}, \mathbf{WX^{<t>}_{(u,m)}} \right]
$$

For each time slice of each record we have:

- matrix $\mathbf{WA}$ is a square matrix of weights with dimensions (*u* x *u*), where *u* is the number of units in the RNN layer (remember: $a^{<t>}$ is an element of the sequence)
- matrix $\mathbf{WX}$ has dimensions (*u* x *m*), where *m* is the number of features (remember: matrix $\mathbf{X}_{(n,m)}$ is the matrix of features, e.g. a word dictionary, or in this case the features of the sound sequence)

This gives the following trainable parameters from the weight matrix $\mathbf{W_{(u,u+m)}}$:

- recurrent weights: $u \cdot u = u^2 = 16^2 = 256$ weights
- input weights: $u \cdot m = 16 \cdot 3 = 48$ weights
- bias terms: 16 (one per unit in the RNN layer)
- dense (output) layer: one unit x 16 input features + 1 bias term = 17

**Total weights**: $256+48+16+16+1=337$

<font color="yellow">**¡Important!: notice that the length of the sequence does not affect the number of parameters (unlike dense layers)**</font>

## RNN model with embeddings

!! Remember !! basic **vocabulary-based input-word representation**:
- **vocabulary** of size `n` (n. of words in the vocabulary)
- n-length vectors with **0's** and (a single) **1** e.g. [0, 0, 0, ..., 1, 0 , 0, ... 0]
- one such vector per word
- **sparse representation** (but huge matrix $\rightarrow$ high dimensional arrays)

E.g.: 15k-word vocabulary, 100 sentences, each sentence max 15-word long: array size = (15k x 15 x 100) $\rightarrow$ over $20 \cdot 10^6$ data points

Word **embeddings** also represent words in an **array**: $\rightarrow$ **continuous vectors**:
- represent any word in *few dimensions* (mostly based on the number of unique words in the text)
- **dense representation** $→$ low dimensional vectors
- not hardcoded but **learned** from the data $\rightarrow$ this is a **key feature** of **embeddings** (like *autoencoders*, automatic extraction of features from the data)

E.g. 15k-word vocabulary, 32 NN units $\rightarrow$ (15k, 32) array (dense) $480\,000$ data points

embedding of a single word over *m* (e.g. 32) dimensions: [-0.054, 0.768, 0.003, 0.832, -0.101, ..., 0.923, -0.509]

<img src="https://github.com/ne1s0n/coding_excercises/blob/master/data/embeddings.png?raw=true">
From: https://towardsdatascience.com/word-embeddings-and-the-chamber-of-secrets-lstm-gru-tf-keras-de3f5c21bf16

- geometric relationship (in the m-dimensional hyperspace) between words in word embeddings can represent **semantic relationships**
- **words closer to each other** (m-dimensional distances) have **stronger relation** compared to words away from each other
- there could be vector 'male to female' which represents the relation between a word and its feminine: may help predict 'king' when 'he' is encountered and 'queen' when 'she' is encountered in the sentence.


Simple example of a `Sequential` model that processes text sequences (e.g. words), embeds each sequence into a 15000-dimensional vector (vocabulary size), then processes the sequence of vectors using a `Simple RNN` layer:

- 100 sentences (text sequences) [n. of records]
- max length of each sequence: 30 words ["time" (longitudinal) dimension]
- vocabulary size: $15\,000$ words [n. of features]

In [ ]:
## parameters
embed_input_size = 15000 ## input vocabulary size
embed_output_size = 32 ## n. of hidden units (nodes)
rnn_units = 16
dense_units = 1 ## e.g. classification

In [ ]:
model = keras.Sequential() ## not unlike usual NN models (including CNN)

model.add(Input((100,))) ## Input sentences of max length 100 words (this does not affect the n. of parameters)
# Add an Embedding layer expecting input vocab of size XXX, and
# output embedding dimension of size ZZZ.
model.add(layers.Embedding(input_dim=embed_input_size, output_dim=embed_output_size)) ## this is unlike dense NN models (model.add(Dense()))

# Add a RNN layer with 16 internal units.
model.add(layers.SimpleRNN(rnn_units))

# Add a Dense layer with 1 units.
model.add(layers.Dense(dense_units))

model.summary()

#### Calculating number of parameters to train

- **embedding layer**: $15\,000 \cdot 32 = 480\,000$
- **RNN layer**: $16 \cdot 16 + 16 \cdot 32 + 16 = 784$
- **output layer**: $16 \cdot 1 + 1 = 17$

**Total weights:** $480\,000 + 784 + 17 = 480\,801$


---
# FROM HERE ON, IT IS ONLY FOR THE VERY BRAVE!
---

### Short-term memory

RNN suffer from short-term memory problems: if a time series is long, RNN have difficulties in carrying information from earlier timepoints over to later timepoints. Specifically, in back propagation RNN experience the **vanishing gradient problem**, i.e. the gradient (values used to update NN weights) shrinks over successive backpropagation steps, and if it becomes too small it won't contribute to learning:

$$
w_{t+1} = w_t - \alpha \cdot \frac{\partial}{\partial w_t}J(w) = 2.1 - 0.1 \cdot 0.001 = 2.0999
$$

(not much of a difference!)

Therefore, RNN can forget what they saw early in the sequence $\rightarrow$ **short-term memory!**

#### Gates in RNN

To address issues with short-term memory, RNN use internal structures (sublayers) called **gates**. Gates can learn which data in a sequence are important to keep or throw away and pass them on along the chain of sequences.
It is s bit like remembering only those words in an ad that struck your memory (e.g. the price, the deadline, the main characteristics)

#### Inside a RNN

- words (or sounds) transformed to vectors
- the RNN processes each sequence of vectors one by one, passing hidden states (units) sequentially to the next steps: in this way, the RNN holds information seen in the each previous step
- the input vector (word) and previous hidden state are combined to form a new vector that has information on the current and previous inputs
- the combined vector goes through the activation function (e.g. `tanh`) and the output is the new hidden state to be combined with the input to be processed by he next unit in the layer
- `tanh` squeashes values from the linear combination of the combined vector values in input (input + hidden state) to the range $[-1,1]$   

Simple RNN (no *gates*, or better only one gate) use much less computational resources than the evolved variants: **LSTM** and **GRU**


#### LSTM

Unlike simple RNN, the operations inside a LSTM cell (sublayer/unit) allow to keep or forget pieces of information. In this way, also information from earlier time steps can make its way to later time steps, reducing the effects of short-term memory. In this journey, information can be added or removed through **gates** (typically **4** in LST layers).
LSTM gates (sublayers) use the **sigmoid** activation function, in $[0,1]$, which permits to 'forget' information by returning 0, or to keep it by returning 1.

- **forget gate**: previous hidden state + input vector $\rightarrow$ `sigmoid` activation: `if sigmoid(h+x) ~ 0 then forget; else if sigmoid(h+x) ~ 1 then keep`
- **input gate(s)**: this typically has two units, one with `sigmoid` and one with `tanh` activation functions; this is where the updating of model weights happens. Each unit receives the previous hidden state + the input vector:
  1. one unit decides on importance (which values to update)
  2. the other unit updates the weights
- **output gate**: this unit combines results from the forget and input gates to produce the output that is passed on to the next times step (next sublayer/unit in the LSTM layer)

### GRU

GRU (gated recurrent unit), compared to LSTM gets rid of `cell state` and use only the `hidden state` to transfer information (LSTM combines cell states and hidden states to pass on information through layers):

- **reset gate**: controls how much past information to forget
- **update gate**: controls what information to throw away and what new information to add
- **output gate**: passes the new hidden state on to the next time step (next unit in the GRU layer)

GRU layers are similar to LSTM layers, but a little faster (fewer tensor operations)


## LSTM layer

Let's calculate the n. of parameters in LSTM layers:
- insight about how LSTM handles time dependent or sequence input data
- model capacity and complexity:
    - handle overfitting or underfitting
    - adjust the number of parameters of each layer

`LSTM` expects input data to be a `tensor` such that:

`[batch_size, timesteps, feature]`

1. `n. of records` (`batch_size`): how many samples (in each batch) during training and testing, e.g. number of sequences to be processed
2. `timesteps`: how many values in a sequence, e.g. in `[4, 7, 8, 4]` there are 4 timesteps (30 words max in our text processing example)
3. `features`: how many dimensions to represent data in one time step; e.g. if each digit value in the sequence is one hot encoded with 9 zero and 1 one then feature is 10, or size of the vocabulary for sentences/words (or output of previous layer)

`LSTM` layers have **4 gates** in their internal structure:

- *forget* gate
- *update* gate
- *candidate* gate
- *output* gate

#### Illustration of a LSTM layer

<img src="https://github.com/kmkarakaya/ML_tutorials/blob/master/images/LSTM_internal2.png?raw=true" width="700">

In [ ]:
embed_input_size = 15000 ## input vocabulary size
embed_output_size = 32 ## n. of hidden units (nodes)
lstm_units = 16
lstm_activation = 'relu'
dense_units = 1
output_acrivation = 'sigmoid'

In [ ]:
model = keras.Sequential() ## not unlike usual NN models (including CNN)

# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 32.
model.add(layers.Embedding(input_dim=embed_input_size, output_dim=embed_output_size)) ## this is unlike dense NN models (model.add(Dense()))

# Add a RNN layer with 16 internal units.
model.add(layers.LSTM(lstm_units, activation = lstm_activation))

# Add a Dense layer with 1 units.
model.add(layers.Dense(dense_units, activation = output_acrivation))

model.summary()

- **i**: 32 (output from the embedding layer)
- **h**: 16 LSTM units
- **g**: number of gates in each LSTM unit

$ [((i+h)×h) + (h)] \cdot g$

The LSTM units (e.g. 16 from the example above) are added to the input units (32, from the example above) and multiplied by the number of LSTM units; the corresponding bias terms are then added (16, one per LSTM unit):

$$
(32+16) \cdot 16 + 16 = 784
$$

This is multiplied by the number of internal gates (4):

$$
784 \cdot 4 = 3\,136
$$

This is how the number of parameters in a LSTM layer is calculated


You can also think in terms of gates: each gate within a unit receives the 32 + 16 input data and has 1 bias term

$(32+16)+1 = 49$

There are 4 gates in a LSTM unit: 49 x 4 = 196

We have 16 LSTM units (nodes) in our LSTM layer: 196 x 16 = 3136

### GRU layers


A similar way of reasoning as with LSTM layers is used also for GRU layers: one important difference is that GRU units have **3 gates** (*candidate*, *update*, *relevance*) instead of 4.

We can therefore calculate the number of trainable parameters as:

$$
[((i+h)×h)+(h)] \cdot g
$$

<img src="https://pluralsight2.imgix.net/guides/c02c6196-7452-4095-9215-c4d57a9dd1a4_1.JPG" width="700">

In [ ]:
embed_input_size = 15000 ## input vocabulary size
embed_output_size = 32 ## n. of hidden units (nodes)
gru_units = 16
gru_activation = 'tanh'
dense_units = 1
output_acrivation = 'sigmoid'

In [ ]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=embed_input_size, output_dim=embed_output_size))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 16)
model.add(layers.GRU(gru_units, activation=gru_activation, return_sequences=True))

model.add(layers.Dense(dense_units, activation=output_acrivation))
model.summary()

$3 \cdot [(32+16) \cdot 16 + 16] = 2\,352$

We see that the calculations give 2352 parameters, which is less that the 2400 parameters computed by `Keras`: there are **48 parameters missing**

This is because some RNN implementations, like `Keras` and `PyTorch` for efficiency reasons overparameterise the model by adding an extra bias term to each GRU gate:

$$
3 \cdot 16 = 48\\
2352 + 48 = 2400
$$

#### To recap GRU layers parameters

1. first, you sum the size of the input vector (original data) and the hidden state (n. of units in the GRU layer): `input_vector` + `hidden_state`: $(h+i)$
2. then you multiply the total imput size by the number of unit in the GRU layer: $h \cdot (h+i)$
3. then you add the bias terms, one per unit: $h \cdot (h+i) + h$
4. then you multiply by the number of gates (typically three in GRU layers): $g \cdot [h \cdot (h+i)+h]$
5. finally, you add additional bias terms, as many as the product of gates and units: $g \cdot [h \cdot (h+i)+h]+g \cdot h$

## A little excercise: train yourself on trainable parameters

Hypothetical example: ECG data (electrocardiography)

- 100 patients
- 60 seconds
- 1 features: voltage

Build a RNN model for heart disease diagnosis:

- 1 simple RNN layer
- 1 GRU layer
- 1 LSTM layer
- 1 output layer

Work out the number of parameters